In [2]:
# This notebook simply gets all the subjects ids from the raw MEG files for the LPP experiment
# and searches for each subject, their MRI anat scan to copy into a folder
# Later on, these scans under the DICOM format will be put into the nii.gz format, using dcm2niix

import pandas as pd
import numpy as np
from pathlib import Path
import subprocess



raw = Path('/home/is153802/neurospin/acquisition/neuromag/data/petit_prince')
scan_path = Path('/home/is153802/neurospin/acquisition/database/Prisma_fit')
output_folder = Path("/home/is153802/workspace_LPP/data/MEG/LPP/anat")
bids_folder = Path("/home/is153802/workspace_LPP/data/MEG/LPP/BIDS")

subj = [str(sub) for sub in raw.iterdir()]
list_scans = []
with open('/home/is153802/code/source-reconstruction/anat_mri.txt', 'r') as fh:
    for line in fh:
        list_scans.append((line))

list_scans = [scan.replace('\n','') for scan in list_scans]

list_scans_subjects = [scan[11:19] for scan in list_scans]

# To generate before the txt file with the anat MRI:
# cd /neurospin/acquisition/database/Prisma_fit 
# find -maxdepth 3 -name '*mprage*' -type d > /home/is153802/code/source-reconstruction/anat_mri.txt
# chmod 777 /home/is153802/code/source-reconstruction/anat_mri.txt


In [3]:
# Cleaning the list of subjects
subjects = []
for sub in subj: 
    index = sub.rfind("petit_prince/")
    subjects.append(sub[index+13:])

# Hardcoded fixes for previous subjects with incorrect names, etc..
# This should not be a problem in the future, thus stay unchanged 
subjects.remove('rt220104_rt220104')
subjects.remove('ml100438_ml100438')
subjects.remove('empty_room')
subjects.append('ml_100438')
subjects.append('rt_220104')
# TODO figure out the ab73???
subjects = [subject.replace('_','') for subject in subjects]

In [4]:
# Intersecting the subjects for LPP and the scans
list_scans_final = [scan for i, scan in enumerate(list_scans) if list_scans_subjects[i] in (subjects)]

In [5]:
# For each subject:
# Get their subset of scans
# Sort it by date, and take the latest
scan_to_move = []

for sub in subjects:
    list_scan_specific = [scan for i, scan in enumerate(list_scans_final) if scan[11:19] == sub]
    # If no scan for the subject, skip it
    if list_scan_specific == []:
        continue
    # print(f'Sub {sub} ! list {list_scan_specific}')
    list_scan_specific.sort(reverse = True)


    scan_to_move.append((list_scan_specific[0]))
scan_to_move = [scan.replace('./','') for scan in scan_to_move]

In [9]:
# Copying the scans to the right folder
for scan in scan_to_move:
    scan_path_specific = scan_path / scan
    sub = scan[9:17]
    if((output_folder/sub).exists()):
        print(f" {output_folder/sub}  exists")
        continue
    else:
        bashCommand = f"mkdir -p {output_folder} "
        process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()
        bashCommand = f"cp -r   {scan_path_specific} {output_folder/sub}"
        process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()

 /home/is153802/workspace_LPP/data/MEG/LPP/anat/pl170230  exists
 /home/is153802/workspace_LPP/data/MEG/LPP/anat/kp210315  exists
 /home/is153802/workspace_LPP/data/MEG/LPP/anat/cc150418  exists
 /home/is153802/workspace_LPP/data/MEG/LPP/anat/mf180020  exists
 /home/is153802/workspace_LPP/data/MEG/LPP/anat/mb180076  exists
 /home/is153802/workspace_LPP/data/MEG/LPP/anat/cm180071  exists
 /home/is153802/workspace_LPP/data/MEG/LPP/anat/kc160388  exists
 /home/is153802/workspace_LPP/data/MEG/LPP/anat/ch180036  exists
 /home/is153802/workspace_LPP/data/MEG/LPP/anat/lg170436  exists
 /home/is153802/workspace_LPP/data/MEG/LPP/anat/ml220421  exists
 /home/is153802/workspace_LPP/data/MEG/LPP/anat/js180232  exists
 /home/is153802/workspace_LPP/data/MEG/LPP/anat/fr190151  exists
 /home/is153802/workspace_LPP/data/MEG/LPP/anat/lq180242  exists
 /home/is153802/workspace_LPP/data/MEG/LPP/anat/hg220389  exists
 /home/is153802/workspace_LPP/data/MEG/LPP/anat/ae140329  exists
 /home/is153802/workspace

In [11]:
scan_to_move


['20220927/pl170230-7754_001/000002_mprage-sag-T1-160sl-iPAT2',
 '20220801/kp210315-7695_001/000002_mprage-sag-T1-160sl',
 '20151207/cc150418-4939_001/000002_mprage-sag-T1-160sl',
 '20180130/mf180020-5904_001/000002_mprage-sag-T1-160sl',
 '20200904/mb180076-6875_001/000025_mprage-sag-T1-160sl-iPat2',
 '20220511/cm180071-7572_002/000019_mprage-sag-T1-160sl-iPAT2',
 '20181009/kc160388-6204_001/000018_mprage-sag-T1-160sl-iPat2',
 '20210602/ch180036-7151_001/000018_mprage-sag-T1-160sl-iPat2',
 '20200915/lg170436-6897_001/000013_mprage-sag-T1-160sl-iPAT2',
 '20221004/eg220435-7769_001/000014_mprage-sag-T1-160sl-iPAT2',
 '20220719/ml220421-7681_001/000014_mprage-sag-T1-160sl',
 '20191217/js180232-6704_001/000022_mprage-sag-T1-160sl',
 '20200130/fr190151-6735_001/000024_mprage-sag-T1-160sl',
 '20190722/cl190429-6553_001/000002_mprage-sag-T1-160sl',
 '20210429/lq180242-7118_001/000018_mprage-sag-T1-160sl-iPAT2',
 '20220614/hg220389-7630_001/000002_mprage-sag-T1-0.9mm-176sl-PF7-8',
 '20200918/a

In [ ]:
# Transforming each folder of DICOM scans to the niix format

dict_id_to_nip = {}
for sub in output_folder.iterdir():
    identifiable = str(sub)[-8:]
    nip = dict_id_to_nip(identifiable)

    dicom_folder_sub = output_folder / identifiable  # where the DICOM files of a sub are stored
    output_bids = bids_folder / f'sub-{nip}/ses-01/anat/'  # where the niix file should be stored (BIDS)

    bashCommand = f"dcm2niix -o {output_bids} -z y -b y \
                    /home/is153802/workspace_LPP/data/MEG/LPP/anat/{dicom_folder_sub}"
    process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()

In [15]:
for sub in output_folder.iterdir():
    print(str(sub)[-8:])

cl190429
hg220389
ml220421
cm180071
eg220435
ml100438
ch180036
ae140329
pl170230
lq180242
kc160388
cl220500
mb180076
fr190151
kp210315
ml180010
cc150418
rt220104
js180232
lg170436
mf180020
